## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,9.14,95,100,6.85,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.25,77,63,14.29,broken clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,73.47,85,43,15.99,scattered clouds
3,3,Luganville,VU,-15.5333,167.1667,79.39,83,48,6.98,scattered clouds
4,4,Changde,CN,29.0464,111.6783,53.51,66,2,3.00,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,81.25,77,63,14.29,broken clouds
3,3,Luganville,VU,-15.5333,167.1667,79.39,83,48,6.98,scattered clouds
6,6,Kapaa,US,22.0752,-159.3190,80.58,76,20,3.00,few clouds
9,9,Guarapari,BR,-20.6667,-40.4975,84.09,86,68,12.24,broken clouds
16,16,Tibiri,NE,13.5627,7.0485,89.69,13,16,8.77,few clouds
20,20,Arlit,NE,18.7369,7.3853,87.26,10,5,6.69,clear sky
30,30,Vaini,TO,-21.2000,-175.2000,75.36,78,20,6.91,few clouds
33,33,Marsh Harbour,BS,26.5412,-77.0636,81.95,72,100,10.16,overcast clouds
38,38,Butaritari,KI,3.0707,172.7902,80.20,71,2,2.66,clear sky
42,42,Mount Pleasant,US,32.7941,-79.8626,82.09,68,1,5.01,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [23]:
# Second check for empty rows
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.25,broken clouds,-0.6000,73.0833,
3,Luganville,VU,79.39,scattered clouds,-15.5333,167.1667,
6,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
9,Guarapari,BR,84.09,broken clouds,-20.6667,-40.4975,
16,Tibiri,NE,89.69,few clouds,13.5627,7.0485,
20,Arlit,NE,87.26,clear sky,18.7369,7.3853,
30,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
33,Marsh Harbour,BS,81.95,overcast clouds,26.5412,-77.0636,
38,Butaritari,KI,80.20,clear sky,3.0707,172.7902,
42,Mount Pleasant,US,82.09,clear sky,32.7941,-79.8626,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))